In [38]:
import pandas as pd
import polars as pl
df = pd.read_csv("/Users/tweber/Data/mosaicatcher_samples_metadata.csv", sep=";")
dt = pl.read_delta("/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65fc33d505358fc517a6e78c/65fc33d505358fc517a6e78d")
dt = dt.to_pandas()
dt[["depictio_run_id", "sample"]].set_index("depictio_run_id").drop_duplicates().to_records().tolist()

[('2019-01-02-HGNWMAFXY', 'SSC11163x02'),
 ('2019-01-02-HGNWMAFXY', 'SSC11168x02'),
 ('2019-01-23-HGNHKAFXY', 'BOM71Gx02'),
 ('2019-01-23-HGNHKAFXY', 'BOMCHIP76Nx02'),
 ('2019-02-01-HH5FHAFXY', 'S149x48hx02'),
 ('2019-02-27-HH5WFAFXY', 'BOMCHIP76N02x01'),
 ('2019-02-27-HH5WFAFXY', 'S14960hx04'),
 ('2019-03-25-HJLHYAFXY', 'BOMH12x01'),
 ('2019-03-25-HJLHYAFXY', 'CB0003x01'),
 ('2019-04-01-HJKNKAFXY', 'BOM65G03x03'),
 ('2019-04-01-HJKNKAFXY', 'BOM71Gx03'),
 ('2019-05-10-HJLFKAFXY', 'HG02059102'),
 ('2019-05-10-HJLFKAFXY', 'SSC1163101'),
 ('2019-05-10-HKY3MAFXY', 'AML1Nx01'),
 ('2019-05-10-HKY3MAFXY', 'BoM92Nx02'),
 ('2019-05-22-HKWWKAFXY', 'BoM712Gx01'),
 ('2019-05-22-HKWWKAFXY', 'WCxHSPCs'),
 ('2019-06-12-HMGM2AFXY', 'HG01352x02'),
 ('2019-06-12-HMGM2AFXY', 'HG02818x02'),
 ('2019-07-23-HMJ7KAFXY', 'ADCr11xCdt1xCdc6'),
 ('2019-07-23-HMJ7KAFXY', 'ADCr11xGFPxmCherry'),
 ('2019-07-23-HMJ7KAFXY', 'GM19434x02'),
 ('2019-07-31-HMHCWAFXY', 'BoM70Nx01'),
 ('2019-07-31-HMHCWAFXY', 'HG04217x02'),


In [39]:
l = list()
df_copy = df.copy()
for e in dt[["depictio_run_id", "sample"]].set_index("depictio_run_id").drop_duplicates().to_records().tolist():
    tmp_df = pd.read_csv(f"/Users/tweber/Data/mosaicatcher-pipeline/{e[0]}/{e[1]}/cell_selection/labels.tsv", sep="\t")
    l.append(tmp_df)
l = pd.concat(l)
pd.merge(df_copy, l[["sample", "cell"]], left_on="sample", right_on="sample").to_csv("/Users/tweber/Data/mosaicatcher-pipeline/mosaicatcher_samples_metadata.csv", sep=";", index=False)

In [1]:
# from gridfs import GridFS
import pymongo
import redis



MONGODB_URL = f"mongodb://0.0.0.0:27018/"
print("MONGODB_URL: ", MONGODB_URL)




client = pymongo.MongoClient(MONGODB_URL)
db = client["depictioDB"]


data_collection = db["data_collections"]
workflow_collection = db["workflows"]
runs_collection = db["runs"]
files_collection = db["files"]
users_collection = db["users"]
# groups_collection = db["groups_collection"]

MONGODB_URL:  mongodb://0.0.0.0:27018/


In [14]:
# Query in files_collection
files_collection.find_one({"file_location": "/Users/tweber/Data/mosaicatcher-pipeline/mosaicatcher_samples_metadata.csv"})
# Show all documents from workflow_collection
# for doc in workflow_collection.find():
#     print(doc)

# remove that file from files_collection
files_collection.delete_one({"file_location": "/Users/tweber/Data/mosaicatcher-pipeline/mosaicatcher_samples_metadata.csv"})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [7]:

# The ID of the data collection you're interested in
from bson import ObjectId


data_collection_id = ObjectId("660c1dabf22783012cc91384")

# Query to retrieve all files belonging to the specified data collection
files = files_collection.find({"data_collection._id": data_collection_id})
for file in files[:10]:
    print(file)

In [48]:
# Constructing the nested dictionary
import collections
from pprint import pprint
nested_dict = collections.defaultdict(lambda: collections.defaultdict(dict))

data_collection_id = ObjectId("65fc33d505358fc517a6e78f")
files = files_collection.find({"data_collection._id": data_collection_id})
for file in files:
    # print(file)
    if file["filename"].endswith(file["data_collection"]["config"]["dc_specific_properties"]["index_extension"]):
        continue
    for wildcard in file["wildcards"]:
        if file["trackId"]:
            nested_dict[file["data_collection"]["_id"]][wildcard["name"]][wildcard["value"]] = file["trackId"]

# Example output
pprint(nested_dict)

defaultdict(<function <lambda> at 0x1335de200>,
            {ObjectId('65fc33d505358fc517a6e78f'): defaultdict(<class 'dict'>,
                                                               {'cell': {'ADCr11xCdt1xCdc6PE20401': '1c30710ff1eaa9ba',
                                                                         'ADCr11xCdt1xCdc6PE20402': '88eda39bde8dd97d',
                                                                         'ADCr11xCdt1xCdc6PE20403': '84eee4a8b86b6a90',
                                                                         'ADCr11xCdt1xCdc6PE20405': 'abe95d4702667fae',
                                                                         'ADCr11xCdt1xCdc6PE20409': 'a3aa1f87a8048740',
                                                                         'ADCr11xCdt1xCdc6PE20410': '119e43ca1422a24c',
                                                                         'ADCr11xCdt1xCdc6PE20411': 'eb31bac4551bd473',
                                 

In [8]:
# id=ObjectId('65fb5bb927b90358513efd08') file_location=PosixPath('/Users/tweber/Data/mosaicatcher-pipeline/2019-09-11-HTN7CAFXY/HG01114x02/plots/JBROWSE/HG01114x02PE20356-SV.bed.gz.tbi') S3_location=None filename='HG01114x02PE20356-SV.bed.gz.tbi' creation_time=datetime.datetime(2024, 1, 29, 10, 5, 44) modification_time=datetime.datetime(2024, 1, 24, 6, 1, 31) data_collection=DataCollection(id=ObjectId('65fb5bb927b90358513efd06'), data_collection_tag='SV_calls', description='BED files containing SV calls generated by MosaiCatcher', config=DataCollectionConfig(type='JBrowse2', regex=Regex(pattern='{cell}-SV.bed.gz', type='file-based', wildcards=[WildcardRegexBase(name='cell', wildcard_regex='.*')]), dc_specific_properties=DCJBrowse2Config(index_extension='tbi', jbrowse_template_location='/Users/tweber/Gits/depictio/CLI_client/configs/mosaicatcher_pipeline/jbrowse2_templates/SV_calls.json'), join=TableJoinConfig(on_columns=['cell'], how='inner', with_dc=['mosaicatcher_stats']))) run_id='2019-09-11-HTN7CAFXY' registration_time=datetime.datetime(2024, 3, 20, 21, 57, 7, 395615) wildcards=[WildcardRegex(name='cell', wildcard_regex='.*', value='HG01114x02PE20356')]
# Find all files with a specific data_collection_id
for file in files_collection.find({"data_collection": {"_id": "660c1dabf22783012cc91384"}}):
    print(file)

In [47]:
import httpx
API_BASE_URL = "http://0.0.0.0:8058"
last_jbrowse_status = httpx.get(f"{API_BASE_URL}/depictio/api/v1/jbrowse/last_status")
last_jbrowse_status = last_jbrowse_status.json()
last_jbrowse_status

{'assembly': 'hg38',
 'loc': '1:0.0..248956422.0',
 'tracks': [],
 'timestamp': 1711033581444,
 'dashboard_id': '1'}